<a href="https://colab.research.google.com/github/atelierkarin/fm-ca-predict/blob/master/Test_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import datetime

In [0]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
max_date = datetime.datetime.strptime('1970/1/1', '%Y/%m/%d')
max_diff = datetime.datetime.today() - max_date

def calc_dob_from_now(dob):
  diff = datetime.datetime.today() - datetime.datetime.strptime(dob, '%Y/%m/%d')
  return diff.total_seconds()

def reformat_internal_goals(app, gl):
  return gl / app if app > 0 else 0

def load_data():
  df_club = pd.read_csv('/content/drive/My Drive/datas/fm-ca-status/club.csv')
  df_competition = pd.read_csv('/content/drive/My Drive/datas/fm-ca-status/competition.csv')
  df_nationality = pd.read_csv('/content/drive/My Drive/datas/fm-ca-status/nationality.csv')
  df_test_data = pd.read_csv('/content/drive/My Drive/datas/fm-ca-status/test_data.csv')
  df_season = pd.read_csv('/content/drive/My Drive/datas/fm-ca-status/season.csv')

  df = df_test_data[:].copy()

  df['DOB_FromNow'] = df.apply(lambda row: calc_dob_from_now(row['DOB']), axis=1)
  df = df.drop(columns='DOB')

  df = pd.merge(df, df_nationality, left_on='Nationality', right_on='Nation Name', how='left').drop(columns='Nation Name').drop(columns='Nationality')
  df = pd.merge(df, df_club, left_on='Club', right_on='Club Name', how='left').drop(columns='Club Name').drop(columns='Club')
  df = pd.merge(df, df_competition, left_on='League', right_on='Competition Name', how='left').drop(columns='League').drop(columns='Competition Name')

  df_ly = df_season[:].copy()
  df_l2y = df_season[:].copy()
  df_l3y = df_season[:].copy()

  df_ly.columns = 'LY ' + df_ly.columns.values
  df_l2y.columns = 'L2Y ' + df_l2y.columns.values
  df_l3y.columns = 'L3Y ' + df_l3y.columns.values

  df_ly_club = df_club[:].copy()
  df_l2y_club = df_club[:].copy()
  df_l3y_club = df_club[:].copy()
  df_ly_competition = df_competition[:].copy()
  df_l2y_competition = df_competition[:].copy()
  df_l3y_competition = df_competition[:].copy()

  df_ly_club.columns = 'LY ' + df_ly_club.columns.values
  df_l2y_club.columns = 'L2Y ' + df_l2y_club.columns.values
  df_l3y_club.columns = 'L3Y ' + df_l3y_club.columns.values
  df_ly_competition.columns = 'LY ' + df_ly_competition.columns.values
  df_l2y_competition.columns = 'L2Y ' + df_l2y_competition.columns.values
  df_l3y_competition.columns = 'L3Y ' + df_l3y_competition.columns.values

  df = pd.merge(df, df_ly, left_on=['LY Year','LY Club','LY League'], right_on=['LY Season','LY Team','LY League'], how='left').drop(columns=['LY Year','LY Season','LY Team'])
  df = pd.merge(df, df_ly_club, left_on='LY Club', right_on='LY Club Name', how='left').drop(columns='LY Club Name').drop(columns='LY Club')
  df = pd.merge(df, df_ly_competition, left_on='LY League', right_on='LY Competition Name', how='left').drop(columns='LY Competition Name').drop(columns='LY League')

  df = pd.merge(df, df_l2y, left_on=['L2Y Year','L2Y Club','L2Y League'], right_on=['L2Y Season','L2Y Team','L2Y League'], how='left').drop(columns=['L2Y Year','L2Y Season','L2Y Team'])
  df = pd.merge(df, df_l2y_club, left_on='L2Y Club', right_on='L2Y Club Name', how='left').drop(columns='L2Y Club Name').drop(columns='L2Y Club')
  df = pd.merge(df, df_l2y_competition, left_on='L2Y League', right_on='L2Y Competition Name', how='left').drop(columns='L2Y Competition Name').drop(columns='L2Y League')

  df = pd.merge(df, df_l3y, left_on=['L3Y Year','L3Y Club','L3Y League'], right_on=['L3Y Season','L3Y Team','L3Y League'], how='left').drop(columns=['L3Y Year','L3Y Season','L3Y Team'])
  df = pd.merge(df, df_l3y_club, left_on='L3Y Club', right_on='L3Y Club Name', how='left').drop(columns='L3Y Club Name').drop(columns='L3Y Club')
  df = pd.merge(df, df_l3y_competition, left_on='L3Y League', right_on='L3Y Competition Name', how='left').drop(columns='L3Y Competition Name').drop(columns='L3Y League')

  df = df.drop(columns=['LY GD','L2Y GD','L3Y GD'])

  df['LY App'] = df['LY App'] / df['LY Pld']
  df['LY Goal'] = df['LY Goal'] / df['LY Pld']
  df['LY W'] = df['LY W'] / df['LY Pld']
  df['LY D'] = df['LY D'] / df['LY Pld']
  df['LY L'] = df['LY L'] / df['LY Pld']
  df['LY GF'] = df['LY GF'] / df['LY Pld']
  df['LY GA'] = df['LY GA'] / df['LY Pld']
  df['LY Pts'] = df['LY Pts'] / df['LY Pld']

  df['L2Y App'] = df['L2Y App'] / df['L2Y Pld']
  df['L2Y Goal'] = df['L2Y Goal'] / df['L2Y Pld']
  df['L2Y W'] = df['L2Y W'] / df['L2Y Pld']
  df['L2Y D'] = df['L2Y D'] / df['L2Y Pld']
  df['L2Y L'] = df['L2Y L'] / df['L2Y Pld']
  df['L2Y GF'] = df['L2Y GF'] / df['L2Y Pld']
  df['L2Y GA'] = df['L2Y GA'] / df['L2Y Pld']
  df['L2Y Pts'] = df['L2Y Pts'] / df['L2Y Pld']

  df['L3Y App'] = df['L3Y App'] / df['L3Y Pld']
  df['L3Y Goal'] = df['L3Y Goal'] / df['L3Y Pld']
  df['L3Y W'] = df['L3Y W'] / df['L3Y Pld']
  df['L3Y D'] = df['L3Y D'] / df['L3Y Pld']
  df['L3Y L'] = df['L3Y L'] / df['L3Y Pld']
  df['L3Y GF'] = df['L3Y GF'] / df['L3Y Pld']
  df['L3Y GA'] = df['L3Y GA'] / df['L3Y Pld']
  df['L3Y Pts'] = df['L3Y Pts'] / df['L3Y Pld']

  df['Nation Rep'] = df['Nation Rep'] / 10000
  df['Club Rep'] = df['Club Rep'] / 10000
  df['Competition Rep'] = df['Competition Rep'] / 200

  df['LY Club Rep'] = df['LY Club Rep'] / 10000
  df['LY Competition Rep'] = df['LY Competition Rep'] / 200

  df['L2Y Club Rep'] = df['L2Y Club Rep'] / 10000
  df['L2Y Competition Rep'] = df['L2Y Competition Rep'] / 200

  df['L3Y Club Rep'] = df['L3Y Club Rep'] / 10000
  df['L3Y Competition Rep'] = df['L3Y Competition Rep'] / 200

  df['Inter. Goals'] = df.apply(lambda row: reformat_internal_goals(row['Inter. Apps'], row['Inter. Goals']), axis=1)
  df['Inter. Apps'] = df['Inter. Apps'] / 255

  df['DOB_FromNow'] = df['DOB_FromNow'] / max_diff.total_seconds()

  df = df.drop(columns=['LY Pld','L2Y Pld','L3Y Pld'])
  
  df = df.set_index('Name')

  return df

In [0]:
df_test = load_data()

In [0]:
from sklearn.externals import joblib

model = joblib.load('/content/drive/My Drive/datas/fm-ca-status/model.joblib')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [0]:
predict_result = model.predict(df_test)

In [0]:
df_test['predict CA'] = predict_result * 200
df_test['predict CA']

Name
福森晃斗     111.020367
深井一希     104.584320
宮澤裕樹     115.883023
鈴木武蔵     110.645225
進藤亮佑     108.399311
土居聖真     120.947853
伊藤翔      117.850094
中村充孝      99.078166
中村駿       98.752364
大久保嘉人    104.765206
平智広       94.322176
前山恭平      81.679613
Name: predict CA, dtype: float64